# Mem0 实现：LLM 代理的可扩展长期记忆（精修版）

本笔记本提供了 Mem0 架构的详细演练和简化实现，重点关注其核心机制，并与全上下文方法进行令牌效率比较。此版本旨在运行期间输出更简洁，将详细日志存储在变量中供后续检查，并以结构化 DataFrame 呈现最终的令牌比较结果。

**场景**：AI 助手帮助用户规划"新营销活动"。

**实现的核心 Mem0 概念（简化版）**：
1. 记忆提取（`ϕ`）
2. 记忆存储（带嵌入的文本事实）
3. 记忆更新逻辑（通过 LLM 实现 ADD、UPDATE、NOOP）
4. 记忆检索（针对查询上下文的语义搜索）

**本版本改进**：
- 减少默认控制台输出
- 通过 `VERBOSE_RAW_RUN` 和 `VERBOSE_MEM0_RUN` 标志按需启用详细日志记录
- 将逐轮详细日志存储在 `raw_run_log` 和 `mem0_run_log` 变量中
- 最终令牌比较结果以 Pandas DataFrame 呈现

## 1. 设置：库、API 配置和辅助函数

### 1.1. 导入库

In [ ]:
# 确保安装以下必要包：
# !pip install openai scikit-learn pandas numpy

In [ ]:
# 导入标准库和第三方库，用于内存、LLM 和数据处理
import os      # 操作系统交互
import json    # JSON 解析/生成（LLM 输出）
import time    # API 调用间的延迟
import uuid    # 唯一内存项 ID
from datetime import datetime  # 时间戳

import numpy as np            # 嵌入向量
import pandas as pd           # DataFrame/表格分析
from openai import OpenAI     # OpenAI 兼容的 LLM/嵌入 API
from sklearn.metrics.pairwise import cosine_similarity  # 嵌入相似度

### 1.2. API 和模型配置

In [ ]:
# --- API 和模型配置 ---

# 从环境变量获取 API 密钥（已在变量 API_KEY 中设置）
# 如果未设置，则抛出错误以防止未经认证的请求
API_KEY = os.getenv("NEBIUS_API_KEY")
if not API_KEY:
    raise ValueError("API 密钥未设置。请设置 NEBIUS_API_KEY 环境变量。")

# 设置 Nebius API 的基础 URL
BASE_URL = "https://api.studio.nebius.com/v1/"

# 指定用于生成响应、提取事实和做出更新决策的 LLM 模型
# 建议使用更强大的模型以获得更好的 Mem0 性能
LLM_MODEL = "deepseek-ai/DeepSeek-V3"

# 指定用于生成文本嵌入的嵌入模型
EMBEDDING_MODEL = "BAAI/bge-multilingual-gemma2"

# 使用指定的基础 URL 和 API 密钥初始化 OpenAI 兼容客户端
client = OpenAI(
    base_url=BASE_URL,
    api_key=API_KEY
)

print(f"OpenAI 客户端已配置。使用 LLM: {LLM_MODEL}, 嵌入: {EMBEDDING_MODEL}")

### 1.3. 全局令牌计数器和日志变量

In [ ]:
# --- 全局令牌计数器 ---
# 跟踪两种方法和每个 Mem0 子任务的令牌使用情况
total_prompt_tokens_raw, total_completion_tokens_raw = 0, 0  # 原始/全上下文方法
total_prompt_tokens_mem0_conversation, total_completion_tokens_mem0_conversation = 0, 0  # Mem0: 对话查询
total_prompt_tokens_mem0_extraction, total_completion_tokens_mem0_extraction = 0, 0      # Mem0: 提取子任务
total_prompt_tokens_mem0_update, total_completion_tokens_mem0_update = 0, 0              # Mem0: 更新子任务

# --- 日志变量 ---
# 存储每次运行的详细日志供后续检查或分析
raw_run_log = []   # 原始/全上下文方法的日志
mem0_run_log = []  # Mem0 方法的日志

def reset_all_token_counters_and_logs():
    """
    重置所有全局令牌计数器和日志列表为零/空。
    确保在运行新实验或比较前有干净的起点。
    """
    global total_prompt_tokens_raw, total_completion_tokens_raw, \
              total_prompt_tokens_mem0_conversation, total_completion_tokens_mem0_conversation, \
              total_prompt_tokens_mem0_extraction, total_completion_tokens_mem0_extraction, \
              total_prompt_tokens_mem0_update, total_completion_tokens_mem0_update, \
              raw_run_log, mem0_run_log

    # 将所有令牌计数器重置为零
    total_prompt_tokens_raw, total_completion_tokens_raw = 0, 0
    total_prompt_tokens_mem0_conversation, total_completion_tokens_mem0_conversation = 0, 0
    total_prompt_tokens_mem0_extraction, total_completion_tokens_mem0_extraction = 0, 0
    total_prompt_tokens_mem0_update, total_completion_tokens_mem0_update = 0, 0

    # 清除所有运行日志
    raw_run_log = []
    mem0_run_log = []
    print("[计数器和日志] 所有令牌计数器和运行日志已重置。")

### 1.4. 核心 LLM 和嵌入辅助函数

In [ ]:
def get_embedding(text_to_embed, verbose=False):
    """
    使用配置的嵌入模型为给定文本生成嵌入向量。
    参数：
        text_to_embed (str): 要嵌入的输入文本
        verbose (bool): 如果为 True，打印错误消息
    返回：
        np.ndarray: 作为 numpy 数组的嵌入向量。失败时返回零向量。
    """
    try:
        # 调用嵌入 API 获取嵌入向量
        response = client.embeddings.create(model=EMBEDDING_MODEL, input=text_to_embed)
        return np.array(response.data[0].embedding)
    except Exception as e:
        # 失败时打印错误（如果 verbose）并返回预期维度的零向量
        if verbose: print(f"[错误] 嵌入失败 '{text_to_embed[:50]}...': {e}。返回零向量。")
        default_embedding_dim = 2560  # 如果模型维度不同，请调整
        return np.zeros(default_embedding_dim)

def get_llm_chat_completion(messages, temperature=0.1, max_tokens=150, verbose=False):
    """
    使用给定的消息和参数调用 LLM 聊天补全 API。
    参数：
        messages (list): 聊天历史的消息字典列表
        temperature (float): LLM 的采样温度
        max_tokens (int): 补全中生成的最大令牌数
        verbose (bool): 如果为 True，打印错误消息
    返回：
        tuple: (response_content, prompt_tokens, completion_tokens)
    """
    try:
        # 调用 LLM 聊天补全 API
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens
        )
        content = response.choices[0].message.content
        prompt_tokens = response.usage.prompt_tokens if response.usage else 0
        completion_tokens = response.usage.completion_tokens if response.usage else 0
        return content, prompt_tokens, completion_tokens
    except Exception as e:
        # 失败时打印错误（如果 verbose）并返回错误消息和零令牌
        if verbose: print(f"[错误] LLM 聊天补全失败: {e}。返回错误消息。")
        return f"错误: LLM 调用失败。{e}", 0, 0

print("嵌入和 LLM 调用的辅助函数已定义。")

## 2. 对话场景：规划营销活动

In [ ]:
# 将会话脚本定义为用户轮次的列表（每个轮次是一个字典）
conversation_script = [
    # 用户陈述活动的主要目标
    {"role": "user", "content": "Hi, let's start planning the 'New Marketing Campaign'. My primary goal is to increase brand awareness by 20%."},
    # 用户指定目标受众
    {"role": "user", "content": "For this campaign, the target audience is young adults aged 18-25."},
    # 用户为社交媒体广告分配初始预算
    {"role": "user", "content": "I want to allocate a budget of $5000 for social media ads for the New Marketing Campaign."},
    # 用户询问活动的主要目标
    {"role": "user", "content": "What's the main goal for the New Marketing Campaign?"},
    # 用户询问目标受众
    {"role": "user", "content": "Who are we targeting for this campaign?"},
    # 用户添加与影响者研究相关的新任务
    {"role": "user", "content": "Let's also consider influencers. Add a task: 'Research potential influencers for the 18-25 demographic' for the New Marketing Campaign."},
    # 用户更新社交媒体广告的预算
    {"role": "user", "content": "Actually, let's increase the social media ad budget for the New Marketing Campaign to $7500."},
    # 用户询问当前社交媒体广告的预算
    {"role": "user", "content": "What's the current budget for social media ads for the New Marketing Campaign?"},
    # 用户询问活动的待处理任务
    {"role": "user", "content": "What tasks do I have pending for this campaign?"},
    # 用户表达对视觉内容的偏好
    {"role": "user", "content": "Also, for the New Marketing Campaign, I prefer visual content for this demographic, like short videos and infographics."}
]

print(f"对话场景已定义，共 {len(conversation_script)} 轮。")

In [ ]:
def classify_input(user_input):
    """
    使用 LLM 将用户输入分类为查询或陈述。
    
    参数：
        user_input (str): 用户的输入消息
    
    返回：
        str: 如果输入是问题则为"query"，否则为"statement"
    """
    # 定义系统提示以指导 LLM 分类规则
    system_prompt = (
        "You are a classifier. "
        "A 'query' is a question or request for information. "
        "A 'statement' is a declaration, instruction, or information that is not a question. "
        "Respond with only one word: either 'query' or 'statement'."
    )
    # 调用 LLM 对输入进行分类
    response = client.chat.completions.create(
        model="microsoft/phi-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Classify the following input as a query or statement: {user_input}"}
        ]
    )
    # 提取并规范化分类结果
    classification = response.choices[0].message.content.strip().lower()
    return classification

In [ ]:
# 遍历会话脚本中的每个轮次
for turn in conversation_script:
    # 使用 LLM 将用户的输入分类为'query'或'statement'
    classification = classify_input(turn["content"])
    # 将分类结果添加到轮次字典中的'type'键下
    turn["type"] = classification
    # 打印输入及其分类以进行验证
    print(f"Input: {turn['content']}, Classification: {classification}")

## 3. 方法 1：原始/全上下文方法

In [ ]:
VERBOSE_RAW_RUN = False  # 设置为 True 以获取详细的逐轮控制台输出

def run_raw_full_context_approach(script):
    """
    使用原始/全上下文方法运行对话：
    - 每个用户轮次都附加到完整的对话历史中
    - LLM 接收整个对话历史以生成每个响应
    - 记录令牌使用情况和响应以供分析
    """
    global total_prompt_tokens_raw, total_completion_tokens_raw, raw_run_log

    print("--- 运行原始/全上下文方法 ---")
    # 使用系统提示初始化对话历史
    current_conversation_history_for_llm = [
        {"role": "system", "content": "You are a helpful assistant. Respond based on the full conversation history."}
    ]

    for i, turn_details in enumerate(script):
        user_message_content = turn_details['content']
        turn_type = turn_details['type']

        if VERBOSE_RAW_RUN:
            print(f"\n--- 原始轮次 {i+1}/{len(script)} (类型: {turn_type}) ---")
        print(f"原始轮次 {i+1} 用户: {user_message_content[:80]}...")

        # 将用户消息添加到对话历史
        current_conversation_history_for_llm.append({"role": "user", "content": user_message_content})

        # 使用完整对话历史调用 LLM
        assistant_response_text, p_tokens, c_tokens = get_llm_chat_completion(
            current_conversation_history_for_llm, max_tokens=150, verbose=VERBOSE_RAW_RUN
        )

        # 更新全局令牌计数器
        total_prompt_tokens_raw += p_tokens
        total_completion_tokens_raw += c_tokens

        print(f"原始轮次 {i+1} 助手: {assistant_response_text[:80]}...")
        if VERBOSE_RAW_RUN:
            print(f"此轮次令牌 (Prompt: {p_tokens}, Completion: {c_tokens})")
            print(f"累计原始令牌 (Prompt: {total_prompt_tokens_raw}, Completion: {total_completion_tokens_raw})")

        # 将助手响应添加到对话历史
        current_conversation_history_for_llm.append({"role": "assistant", "content": assistant_response_text})

        # 记录轮次详情供后续分析
        raw_run_log.append({
            "turn": i + 1,
            "type": turn_type,
            "user_content": user_message_content,
            "assistant_response": assistant_response_text,
            "prompt_tokens_turn": p_tokens,
            "completion_tokens_turn": c_tokens,
            "cumulative_prompt_tokens": total_prompt_tokens_raw,
            "cumulative_completion_tokens": total_completion_tokens_raw
        })

        time.sleep(0.2)  # 减少睡眠时间，如果速率限制有问题请调整

    print("\n--- 原始/全上下文方法摘要 ---")
    print(f"总提示令牌: {total_prompt_tokens_raw}")
    print(f"总补全令牌: {total_completion_tokens_raw}")
    print(f"原始方法总令牌数: {total_prompt_tokens_raw + total_completion_tokens_raw}")

## 4. 方法 2：Mem0 实现

### 4.1. 记忆项和记忆存储类

In [ ]:
class MemoryItem:
    def __init__(self, text_content, source_turn_indices_list, verbose_embedding=False):
        """
        表示具有文本、嵌入和元数据的单个记忆项。
        
        参数：
            text_content (str): 要存储在记忆中的内容
            source_turn_indices_list (list): 贡献于此记忆的对话轮次索引列表
            verbose_embedding (bool): 如果为 True，打印嵌入错误
        """
        self.id = str(uuid.uuid4())  # 记忆项的唯一标识符
        self.text = text_content  # 实际记忆内容
        self.embedding = get_embedding(text_content, verbose=verbose_embedding)  # 嵌入向量
        self.creation_timestamp = datetime.now()  # 记忆创建时间
        self.last_accessed_timestamp = self.creation_timestamp  # 最后访问时间
        self.access_count = 0  # 访问次数
        self.source_turn_indices = list(source_turn_indices_list)  # 来源轮次用于溯源

    def __repr__(self):
        # 用于调试的字符串表示
        return (f"MemoryItem(id={self.id}, text='{self.text[:60]}...', "
                f"created={self.creation_timestamp.strftime('%H:%M:%S')}, accessed={self.access_count})")

    def mark_accessed(self):
        # 当记忆被访问时更新访问元数据
        self.last_accessed_timestamp = datetime.now()
        self.access_count += 1

In [ ]:
class MemoryStore:
    def __init__(self, verbose_ops=False):
        """
        存储和管理多个 MemoryItem 实例。
        
        参数：
            verbose_ops (bool): 如果为 True，打印操作的调试信息
        """
        self.memories = {}  # 存储 memory_id -> MemoryItem 的字典
        self.verbose = verbose_ops
        if self.verbose:
            print("[MemoryStore] 初始化了一个空的记忆存储。")

    def add_memory_item(self, memory_item_instance):
        """
        向存储中添加新的 MemoryItem。
        
        参数：
            memory_item_instance (MemoryItem): 要添加的记忆项
        """
        self.memories[memory_item_instance.id] = memory_item_instance
        if self.verbose:
            print(f"[MemoryStore] 已添加: '{memory_item_instance.text[:70]}' (ID: {memory_item_instance.id})")

    def get_memory_item_by_id(self, memory_id):
        """
        通过 ID 检索 MemoryItem 并标记为已访问。
        
        参数：
            memory_id (str): 记忆项的 ID
        
        返回：
            MemoryItem 或 None: 如果找到则返回记忆项，否则返回 None
        """
        if memory_id in self.memories:
            self.memories[memory_id].mark_accessed()  # 标记为已访问
            return self.memories[memory_id]
        return None

    def update_existing_memory_item(self, memory_id, new_text_content, contributing_turn_indices):
        """
        更新现有记忆项的文本和嵌入。
        
        参数：
            memory_id (str): 要更新的记忆 ID
            new_text_content (str): 新的文本内容
            contributing_turn_indices (list): 要添加的额外轮次索引
        
        返回：
            bool: 如果更新成功则为 True，否则为 False
        """
        if memory_id in self.memories:
            memory_to_update = self.memories[memory_id]
            original_text_preview = memory_to_update.text[:70]
            # 更新文本和嵌入
            memory_to_update.text = new_text_content
            memory_to_update.embedding = get_embedding(new_text_content, verbose=self.verbose)
            memory_to_update.creation_timestamp = datetime.now()
            # 添加新的贡献轮次索引（如果尚未存在）
            for turn_idx in contributing_turn_indices:
                if turn_idx not in memory_to_update.source_turn_indices:
                    memory_to_update.source_turn_indices.append(turn_idx)
            memory_to_update.mark_accessed()
            if self.verbose:
                print(f"[MemoryStore] 已更新 ID {memory_id}: 从 '{original_text_preview}' 到 '{new_text_content[:70]}'")
            return True
        if self.verbose:
            print(f"[MemoryStore] 更新失败: ID {memory_id} 未找到。")
        return False

    def find_semantically_similar_memories(self, query_text_embedding, top_s_results=3, similarity_threshold=0.5):
        """
        找到与查询嵌入最相似的记忆。
        
        参数：
            query_text_embedding (np.ndarray): 查询文本的嵌入
            top_s_results (int): 返回的最大结果数
            similarity_threshold (float): 考虑的最小相似度分数
        
        返回：
            list: (MemoryItem, similarity_score) 元组的列表
        """
        # 如果没有记忆或查询嵌入无效，则返回空
        if not self.memories or query_text_embedding is None or query_text_embedding.size == 0:
            return []
        # 过滤掉嵌入无效的记忆
        valid_memories_for_similarity = [
            (mid, self.memories[mid].embedding) for mid in self.memories
            if self.memories[mid].embedding is not None and 
               self.memories[mid].embedding.size > 0 and 
               np.any(self.memories[mid].embedding)  # 确保不全为零
        ]
        if not valid_memories_for_similarity:
            return []
        valid_memory_ids = [item[0] for item in valid_memories_for_similarity]
        valid_memory_embeddings = np.array([item[1] for item in valid_memories_for_similarity])
        # 处理单个有效记忆的情况
        if valid_memory_embeddings.ndim == 1:
            valid_memory_embeddings = valid_memory_embeddings.reshape(1, -1)
        if query_text_embedding.ndim == 1:
            query_text_embedding = query_text_embedding.reshape(1, -1)

        # 计算查询与所有有效记忆之间的余弦相似度
        similarities_vector = cosine_similarity(query_text_embedding, valid_memory_embeddings)[0]
        sorted_similarity_indices = np.argsort(similarities_vector)[::-1]
        retrieved_similar_memories = []
        # 收集高于阈值的顶部结果
        for i in range(min(top_s_results, len(sorted_similarity_indices))):
            idx = sorted_similarity_indices[i]
            similarity_score = similarities_vector[idx]
            if similarity_score >= similarity_threshold:
                retrieved_similar_memories.append((self.memories[valid_memory_ids[idx]], similarity_score))
            else:
                break
        return retrieved_similar_memories

    def clear_store(self):
        """
        清除存储中的所有记忆。
        """
        self.memories = {}
        if self.verbose:
            print("[MemoryStore] 记忆存储已清除。")

# 实例化记忆存储（设置 verbose_ops=True 以获取调试输出）
mem0_memory_store = MemoryStore(verbose_ops=False)

### 4.2. 记忆提取函数（`ϕ`）

In [ ]:
def mem0_extract_salient_facts_from_turn(current_user_statement_text, recent_turns_window_text, current_turn_index_in_script, verbose=False):
    """
    使用 LLM 从用户陈述中提取简洁、自包含的陈述性事实，
    考虑最近的对话上下文。返回事实字符串列表。
    
    参数：
        current_user_statement_text (str): 用户的当前陈述
        recent_turns_window_text (str): 作为文本的最近对话上下文
        current_turn_index_in_script (int): 当前轮次在脚本中的索引
        verbose (bool): 如果为 True，打印调试信息
    
    返回：
        list: 提取的事实字符串列表
    """
    global total_prompt_tokens_mem0_extraction, total_completion_tokens_mem0_extraction

    # 为 LLM 构建提取提示
    extraction_prompt_template = f"""
    You are an AI expert in extracting key information from dialogue.
    Analyze 'New User Statement' in context of 'Recent Conversation Context'.
    Extract concise, self-contained, declarative facts representing new, important user-provided information from 'New User Statement'.
    Each fact: complete sentence. Focus: user's goals, plans, preferences, decisions, key entities.
    Avoid: questions, acknowledgements, fluff. If statement updates info, extract updated fact. Do NOT infer.
    Recent Conversation Context:
    ---BEGIN CONTEXT---
    {recent_turns_window_text if recent_turns_window_text else "(No prior context)"}
    ---END CONTEXT---
    New User Statement to Process: "{current_user_statement_text}"
    Output ONLY a valid JSON list of strings (facts). E.g.: ["Fact 1.", "Fact 2."]. Empty list [] if no new salient facts.
    Extracted Facts (JSON list):
    """
    # 为 LLM 调用准备消息
    extraction_messages = [
        {"role": "system", "content": "Expert extraction AI. Output ONLY valid JSON list of facts."},
        {"role": "user", "content": extraction_prompt_template}
    ]

    # 调用 LLM 提取事实
    llm_extraction_response_text, p_tokens, c_tokens = get_llm_chat_completion(
        extraction_messages, temperature=0.0, max_tokens=250, verbose=verbose
    )

    # 更新全局令牌计数器
    total_prompt_tokens_mem0_extraction += p_tokens
    total_completion_tokens_mem0_extraction += c_tokens

    if verbose:
        print(f"[Extractor LLM] 原始输出: {llm_extraction_response_text}")

    # 尝试将 LLM 的输出解析为字符串的 JSON 列表
    try:
        json_start_index = llm_extraction_response_text.find('[')
        json_end_index = llm_extraction_response_text.rfind(']')
        if json_start_index != -1 and json_end_index != -1 and json_end_index > json_start_index:
            json_string_candidate = llm_extraction_response_text[json_start_index : json_end_index+1]
            parsed_facts_list = json.loads(json_string_candidate)
            if isinstance(parsed_facts_list, list) and all(isinstance(fact, str) for fact in parsed_facts_list):
                if verbose or len(parsed_facts_list) > 0:
                    print(f"[Extractor LLM] 解析了 {len(parsed_facts_list)} 个事实。")
                return parsed_facts_list
            if verbose:
                print(f"[Extractor LLM] 警告: 解析的 JSON 不是字符串列表: {parsed_facts_list}。返回 []。")
        elif verbose:
            print(f"[Extractor LLM] 警告: 在 '{llm_extraction_response_text}' 中没有有效的 JSON 列表括号。返回 []。")
    except Exception as e:
        if verbose:
            print(f"[Extractor LLM] 解析 JSON 错误: {e}。响应: '{llm_extraction_response_text}'。返回 []。")
    return []

### 4.3. 记忆更新逻辑（ADD、UPDATE、NOOP）

In [ ]:
# 用于更新决策的相似记忆数量
S_SIMILAR_MEMORIES_FOR_UPDATE_DECISION = 3

def mem0_decide_memory_operation_with_llm(candidate_fact_text, similar_existing_memories_list, verbose=False):
    """
    使用 LLM 决定是 ADD、UPDATE 还是 NOOP 候选事实到记忆中，
    基于其与现有记忆的相似性。
    
    参数：
        candidate_fact_text (str): 从用户输入中提取的新事实
        similar_existing_memories_list (list): (MemoryItem, similarity_score) 元组列表
        verbose (bool): 如果为 True，打印调试信息
    
    返回：
        dict: 带有操作决策的 JSON 对象，例如，
              {"operation": "ADD"} 或
              {"operation": "UPDATE", "target_memory_id": "ID", "updated_memory_text": "Text"} 或
              {"operation": "NOOP"}
    """
    global total_prompt_tokens_mem0_update, total_completion_tokens_mem0_update

    # 准备描述相似记忆的提示段（如果有）
    similar_memories_prompt_segment = "No highly similar memories found."
    if similar_existing_memories_list:
        formatted_list = [
            f"  {i+1}. ID: {mem.id}, Sim: {sim_score:.4f}, Text: '{mem.text}'"
            for i, (mem, sim_score) in enumerate(similar_existing_memories_list)
        ]
        similar_memories_prompt_segment = "Existing Similar Memories:\n" + "\n".join(formatted_list)
    
    # 为 LLM 构建更新决策提示
    formatted_update_decision_prompt = f"""
    AI Memory Consolidation Module.
    Task: Integrate 'New Candidate Fact' into memory store.
    New Candidate Fact: "{candidate_fact_text}"
    {similar_memories_prompt_segment}
    Decide ONE operation: ADD, UPDATE, or NOOP.
    Rules:
    1. ADD: If new info, not covered by similar memories (or no similar found).
    2. UPDATE: If fact corrects, makes current, or adds essential detail to ONE similar memory, superseding it. Specify 'target_memory_id' (from list) and 'updated_memory_text' (usually New Candidate Fact text).
    3. NOOP: If redundant or no new substantive value over existing similar memories.
    Output STRICTLY JSON: {{ "operation": "ADD" }} OR {{ "operation": "UPDATE", "target_memory_id": "ID", "updated_memory_text": "Text" }} OR {{ "operation": "NOOP" }}
    Decision (JSON object):
    """
    # 为 LLM 调用准备消息
    update_decision_messages = [
        {"role": "system", "content": "Expert memory AI. Output ONLY valid JSON decision as instructed."},
        {"role": "user", "content": formatted_update_decision_prompt}
    ]

    # 调用 LLM 获取更新决策
    llm_decision_response_text, p_tokens, c_tokens = get_llm_chat_completion(
        update_decision_messages, temperature=0.0, max_tokens=200, verbose=verbose
    )

    # 更新全局令牌计数器
    total_prompt_tokens_mem0_update += p_tokens
    total_completion_tokens_mem0_update += c_tokens

    if verbose:
        print(f"[Updater LLM] 原始输出: {llm_decision_response_text}")

    # 尝试将 LLM 的输出解析为 JSON 对象
    try:
        json_start_index = llm_decision_response_text.find('{')
        json_end_index = llm_decision_response_text.rfind('}')
        if json_start_index != -1 and json_end_index != -1 and json_end_index > json_start_index:
            json_string_candidate = llm_decision_response_text[json_start_index : json_end_index+1]
            parsed_decision_json = json.loads(json_string_candidate)
            op = parsed_decision_json.get("operation")
            # 检查有效的操作类型和必填字段
            if op in ["ADD", "NOOP"]:
                if verbose or op == "NOOP":
                    print(f"[Updater LLM] 解析的决策: {op}")
                return parsed_decision_json
            elif op == "UPDATE" and "target_memory_id" in parsed_decision_json and "updated_memory_text" in parsed_decision_json:
                if verbose:
                    print(f"[Updater LLM] 解析的决策: {op}")
                return parsed_decision_json
            if verbose:
                print(f"[Updater LLM] 警告: 无效的决策结构: {parsed_decision_json}。默认为 ADD。")
        elif verbose:
            print(f"[Updater LLM] 警告: 在 '{llm_decision_response_text}' 中没有有效的 JSON 对象括号。默认为 ADD。")
    except Exception as e:
        if verbose:
            print(f"[Updater LLM] 解析 JSON 错误: {e}。响应: '{llm_decision_response_text}'。默认为 ADD。")
    # 默认回退: ADD 操作
    return {"operation": "ADD"}

### 4.4. 协调记忆提取和更新

In [ ]:
def mem0_process_user_statement_for_memory(user_statement_text, recent_turns_context_text, memory_store_instance, current_turn_idx, turn_log_entry, verbose=False):
    """
    协调从用户陈述中提取显著事实的过程，
    确定每个事实的记忆操作（ADD、UPDATE、NOOP），并相应地更新记忆存储。
    
    参数：
        user_statement_text (str): 要处理的用户陈述
        recent_turns_context_text (str): 用于提取的最近对话上下文
        memory_store_instance (MemoryStore): 要更新的记忆存储
        current_turn_idx (int): 当前轮次在脚本中的索引
        turn_log_entry (dict): 此轮次的日志条目（用于详细日志记录）
        verbose (bool): 如果为 True，打印详细的调试信息
    """
    if verbose:
        print(f"[MemoryOrchestrator] 处理: '{user_statement_text[:60]}...' (轮次: {current_turn_idx})")
    turn_log_entry['extraction_details'] = []
    turn_log_entry['update_details'] = []

    # 步骤 1: 使用 LLM 提取器从用户陈述中提取候选事实
    candidate_facts_list = mem0_extract_salient_facts_from_turn(
        user_statement_text, recent_turns_context_text, current_turn_idx, verbose=verbose
    )
    turn_log_entry['extracted_facts_raw'] = list(candidate_facts_list)
    if not candidate_facts_list:
        if verbose:
            print("[MemoryOrchestrator] 没有提取到候选事实。")
        return
    if verbose or len(candidate_facts_list) > 0:
        print(f"[MemoryOrchestrator] 提取了 {len(candidate_facts_list)} 个事实。")

    # 步骤 2: 对于每个提取的事实，确定适当的记忆操作
    for fact_idx, individual_fact_text in enumerate(candidate_facts_list):
        extraction_detail = {
            "candidate_fact": individual_fact_text,
            "similar_memories_checked": [],
            "llm_decision": None
        }
        if verbose:
            print(f"\n[MemoryOrchestrator] -> 处理事实 {fact_idx+1}: '{individual_fact_text[:60]}...'")
        # 为候选事实生成嵌入
        candidate_fact_embedding = get_embedding(individual_fact_text, verbose=verbose)
        # 在存储中查找相似的记忆
        similar_memories_found = memory_store_instance.find_semantically_similar_memories(
            candidate_fact_embedding, top_s_results=S_SIMILAR_MEMORIES_FOR_UPDATE_DECISION
        )
        if similar_memories_found:
            if verbose:
                print(f"[MemoryOrchestrator]    找到 {len(similar_memories_found)} 个相似记忆。")
            for mem, sim_score in similar_memories_found:
                extraction_detail['similar_memories_checked'].append({
                    'id': mem.id,
                    'text': mem.text,
                    'similarity': sim_score
                })
                if verbose:
                    print(f"[MemoryOrchestrator]     ID: {mem.id}, 相似度: {sim_score:.2f}, 文本: '{mem.text[:50]}...'")
        elif verbose:
            print("[MemoryOrchestrator]    没有找到高度相似的记忆。")

        # 步骤 3: 使用 LLM 决定 ADD、UPDATE 或 NOOP 操作
        llm_decision_json = mem0_decide_memory_operation_with_llm(
            individual_fact_text, similar_memories_found, verbose=verbose
        )
        operation_to_perform = llm_decision_json.get("operation")
        extraction_detail['llm_decision'] = llm_decision_json
        print(f"[MemoryOrchestrator] 事实 '{individual_fact_text[:30]}...': LLM 决策 -> {operation_to_perform}")
        turn_log_entry['update_details'].append(extraction_detail)  # 在执行前记录

        # 步骤 4: 执行决定的记忆操作
        if operation_to_perform == "ADD":
            # 向存储添加新记忆项
            new_memory_item = MemoryItem(
                text_content=individual_fact_text,
                source_turn_indices_list=[current_turn_idx],
                verbose_embedding=verbose
            )
            memory_store_instance.add_memory_item(new_memory_item)
        elif operation_to_perform == "UPDATE":
            # 更新现有记忆项
            target_memory_id = llm_decision_json.get("target_memory_id")
            updated_fact_text = llm_decision_json.get("updated_memory_text")
            # 检查目标 ID 是否合理（在相似记忆中）
            is_plausible_target = any(mem.id == target_memory_id for mem, _ in similar_memories_found)
            if target_memory_id and updated_fact_text and is_plausible_target:
                memory_store_instance.update_existing_memory_item(
                    target_memory_id, updated_fact_text, [current_turn_idx]
                )
            elif target_memory_id and updated_fact_text:
                # 即使不在相似列表中也要尝试更新；如果失败则回退到添加
                if verbose:
                    print(f"[MemoryOrchestrator] 警告: UPDATE target_id '{target_memory_id}' 不在相似列表中。尝试更新。")
                if not memory_store_instance.update_existing_memory_item(
                    target_memory_id, updated_fact_text, [current_turn_idx]
                ):
                    if verbose:
                        print(f"[MemoryOrchestrator] 更新失败。添加为新项。")
                    memory_store_instance.add_memory_item(MemoryItem(individual_fact_text, [current_turn_idx], verbose))
            else:
                # 格式错误的 UPDATE，回退到添加为新项
                if verbose:
                    print(f"[MemoryOrchestrator] 警告: UPDATE 格式错误/不合理。添加为新项。")
                memory_store_instance.add_memory_item(MemoryItem(individual_fact_text, [current_turn_idx], verbose))
        elif operation_to_perform == "NOOP":
            # 对于冗余事实不做任何操作
            if verbose:
                print(f"[MemoryOrchestrator]    NOOP 事实: '{individual_fact_text[:70]}...'")
        else:
            # 回退: 如果未知操作则视为 ADD
            if verbose:
                print(f"[MemoryOrchestrator] 警告: 未知操作 '{operation_to_perform}'。默认为 ADD。")
            memory_store_instance.add_memory_item(MemoryItem(individual_fact_text, [current_turn_idx], verbose))
        time.sleep(0.1)  # 小延迟以避免 API 速率限制

### 4.5. 记忆检索以回答用户查询

In [ ]:
K_MEMORIES_TO_RETRIEVE_FOR_QUERY = 3

def mem0_retrieve_and_format_memories_for_llm_query(
    user_query_text, memory_store_instance, turn_log_entry, 
    top_k_results=K_MEMORIES_TO_RETRIEVE_FOR_QUERY, verbose=False
):
    """
    检索用户查询的 top-k 语义相关记忆，并将其格式化为 LLM 输入。
    
    参数：
        user_query_text (str): 用户的查询
        memory_store_instance (MemoryStore): 要搜索的记忆存储
        turn_log_entry (dict): 此轮次的日志条目（用于详细日志记录）
        top_k_results (int): 要检索的顶部相关记忆数量
        verbose (bool): 如果为 True，打印调试信息
    
    返回：
        str: 相关记忆的格式化字符串供 LLM 使用，如果没有找到则返回消息
    """
    # 初始化检索到的记忆日志
    turn_log_entry['retrieved_memories_for_query'] = []

    # 如果查询为空或记忆存储为空，则提前返回
    if not user_query_text.strip() or not memory_store_instance.memories:
        return "(No relevant memories in store or query empty.)"
        
    # 为用户查询生成嵌入
    query_embedding = get_embedding(user_query_text, verbose=verbose)

    # 检索 top-k 语义相似的记忆
    retrieved_memories_with_scores = memory_store_instance.find_semantically_similar_memories(
        query_embedding, top_s_results=top_k_results
    )

    # 如果没有找到相关记忆，则返回消息
    if not retrieved_memories_with_scores:
        return "(No relevant memories found for this query.)"
    
    # 为 LLM 输入格式化检索到的记忆
    formatted_memories_string = "Based on my memory, here's relevant information:\n"
    for i, (mem_item, similarity_score) in enumerate(retrieved_memories_with_scores):
        memory_store_instance.get_memory_item_by_id(mem_item.id)  # 标记为已访问
        formatted_memories_string += f"  {i+1}. {mem_item.text} (Similarity: {similarity_score:.3f})\n"
        # 记录检索到的记忆详情
        turn_log_entry['retrieved_memories_for_query'].append({
            'id': mem_item.id, 
            'text': mem_item.text, 
            'similarity': similarity_score
        })
    if verbose:
        print(f"[Retriever] 为 LLM 格式化的记忆: \n{formatted_memories_string}")
    return formatted_memories_string.strip()

### 4.6. 主函数：运行 Mem0 驱动的对话

In [ ]:
# 控制 Mem0 运行的详细程度（设置为 True 获取详细输出）
VERBOSE_MEM0_RUN = False

# 用于记忆提取上下文的最近轮次数
M_RECENT_RAW_TURNS_FOR_EXTRACTION_CONTEXT = 3

# 为 LLM 上下文保留的短期聊天历史中的最近轮次数
SHORT_TERM_CHAT_HISTORY_WINDOW = 2

def run_mem0_approach_conversation(script, memory_store_instance):
    """
    使用 Mem0 记忆驱动的方法运行对话。
    处理用户陈述（用于记忆提取/更新）和查询（用于记忆检索/回答）。
    跟踪令牌使用情况并记录每轮的详细信息。
    
    参数：
        script (list): 字典列表，每个字典表示带有 'content' 和 'type' 的用户轮次
        memory_store_instance (MemoryStore): 用于此次运行的记忆存储
    """
    global total_prompt_tokens_mem0_conversation, total_completion_tokens_mem0_conversation, mem0_run_log

    # 重置记忆存储并设置详细程度
    memory_store_instance.clear_store()
    memory_store_instance.verbose = VERBOSE_MEM0_RUN

    print("--- 运行 Mem0 驱动的对话方法 ---")

    # 用于提取上下文的原始日志（用于事实提取）
    raw_conversation_log_for_extraction_context = []

    # 用于 LLM 上下文的短期聊天历史（系统提示 + 最后 N 轮）
    current_short_term_llm_chat_history = [
        {"role": "system", "content": "Helpful AI assistant. Use general knowledge and 'Relevant Information from Memory' to answer concisely."}
    ]

    for turn_index, turn_data in enumerate(script):
        user_message_content = turn_data['content']
        turn_type = turn_data['type']
        turn_log_entry = {
            "turn": turn_index + 1,
            "type": turn_type,
            "user_content": user_message_content
        }

        print(f"\n--- Mem0 轮次 {turn_index + 1}/{len(script)} ({turn_type}) ---")
        print(f"用户: {user_message_content[:80]}...")

        assistant_response_text = "(Ack/Internal Processing)"

        # 处理用户陈述（添加/更新记忆）
        if turn_type == 'statement' or turn_type == 'statement_update':
            # 获取用于提取上下文的最近轮次
            start_idx = max(0, len(raw_conversation_log_for_extraction_context) - M_RECENT_RAW_TURNS_FOR_EXTRACTION_CONTEXT)
            recent_turns_text = "\n".join(raw_conversation_log_for_extraction_context[start_idx:])

            # 提取事实并更新记忆存储
            mem0_process_user_statement_for_memory(
                user_message_content,
                recent_turns_text,
                memory_store_instance,
                turn_index,
                turn_log_entry,
                verbose=VERBOSE_MEM0_RUN
            )

            # 提供确认响应
            assistant_response_text = "Okay, noted." if turn_type == 'statement' else "Okay, updated."
            print(f"助手 (确认): {assistant_response_text}")

            # 记录响应和令牌使用情况（确认无需 LLM 调用）
            turn_log_entry['assistant_response_conversational'] = assistant_response_text
            turn_log_entry['prompt_tokens_conversational_turn'] = 0
            turn_log_entry['completion_tokens_conversational_turn'] = 0

        # 处理用户查询（检索记忆并回答）
        elif turn_type == 'query':
            if VERBOSE_MEM0_RUN:
                print(f"[Mem0 Run] 处理查询: '{user_message_content}'")

            # 为查询检索相关记忆
            retrieved_memories_text = mem0_retrieve_and_format_memories_for_llm_query(
                user_message_content,
                memory_store_instance,
                turn_log_entry,
                verbose=VERBOSE_MEM0_RUN
            )

            # 准备 LLM 输入: 短期聊天 + 用户查询 + 相关记忆
            messages_for_llm = list(current_short_term_llm_chat_history)
            messages_for_llm.append({
                "role": "user",
                "content": f"User Query: '{user_message_content}'\n\nRelevant Info from Memory:\n{retrieved_memories_text}"
            })

            # 从 LLM 获取助手响应
            assistant_response_text, p_tokens, c_tokens = get_llm_chat_completion(
                messages_for_llm,
                max_tokens=120,
                verbose=VERBOSE_MEM0_RUN
            )

            # 更新全局令牌计数器
            total_prompt_tokens_mem0_conversation += p_tokens
            total_completion_tokens_mem0_conversation += c_tokens

            print(f"助手: {assistant_response_text[:80]}...")
            if VERBOSE_MEM0_RUN:
                print(f"  查询响应的令牌 (P: {p_tokens}, C: {c_tokens})")

            # 记录响应和令牌使用情况
            turn_log_entry['assistant_response_conversational'] = assistant_response_text
            turn_log_entry['prompt_tokens_conversational_turn'] = p_tokens
            turn_log_entry['completion_tokens_conversational_turn'] = c_tokens

        # 更新提取上下文日志（用于未来轮次的事实提取）
        raw_conversation_log_for_extraction_context.append(f"T{turn_index+1} U: {user_message_content}")
        raw_conversation_log_for_extraction_context.append(f"T{turn_index+1} A: {assistant_response_text}")

        # 为 LLM 上下文更新短期聊天历史
        current_short_term_llm_chat_history.extend([
            {"role": "user", "content": user_message_content},
            {"role": "assistant", "content": assistant_response_text}
        ])
        # 截断聊天历史以仅保留最近的 N 轮（加上系统提示）
        if len(current_short_term_llm_chat_history) > (1 + SHORT_TERM_CHAT_HISTORY_WINDOW * 2):
            current_short_term_llm_chat_history = [current_short_term_llm_chat_history[0]] + \
                current_short_term_llm_chat_history[-(SHORT_TERM_CHAT_HISTORY_WINDOW*2):]

        # 记录此轮次后的记忆存储大小
        turn_log_entry['mem_store_size_after_turn'] = len(memory_store_instance.memories)
        mem0_run_log.append(turn_log_entry)

        if VERBOSE_MEM0_RUN:
            print(f"  当前 Mem0 存储大小: {len(memory_store_instance.memories)}")

        # 小延迟以避免 API 速率限制
        time.sleep(0.2)

    print("\n--- Mem0 驱动的对话方法摘要（全局计数器）---")
    # 如果需要，可以通过全局计数器获取最终令牌总数

    if VERBOSE_MEM0_RUN:
        print("\n--- Mem0 记忆存储的最终内容 ---")
        if memory_store_instance.memories:
            for mem_id, mem_item in memory_store_instance.memories.items():
                print(f"  ID: {mem_id}\n    文本: '{mem_item.text}'\n    创建: {mem_item.creation_timestamp.strftime('%H:%M:%S')}, 访问: {mem_item.access_count}, 最后: {mem_item.last_accessed_timestamp.strftime('%H:%M:%S')}, 来源: {mem_item.source_turn_indices}")
        else:
            print("  Mem0 记忆存储为空。")

## 5. 执行和比较分析

In [ ]:
# --- 比较分析的主要执行块 ---
print("开始原始方法与 Mem0 方法的比较分析。\n")

# --- 运行原始/全上下文方法 ---
print("步骤 1: 运行原始/全上下文方法...")

# 重置所有全局令牌计数器和日志以确保干净的运行
reset_all_token_counters_and_logs()

# 设置原始方法的详细程度（设置为 True 获取详细日志）
VERBOSE_RAW_RUN = False

# 使用对话脚本运行原始/全上下文方法
run_raw_full_context_approach(conversation_script)

# 捕获原始方法的最终令牌计数
final_raw_prompt_tokens = total_prompt_tokens_raw
final_raw_completion_tokens = total_completion_tokens_raw
final_raw_total_tokens = final_raw_prompt_tokens + final_raw_completion_tokens

print("原始方法运行完成。")

In [ ]:
# --- 运行 Mem0 驱动的方法 ---
print("步骤 2: 运行 Mem0 驱动的对话方法...")

# 注意: 如果独立运行此单元格，请确保 Mem0 计数器和日志已重置以进行干净的运行。
# 如果之前调用了 reset_all_token_counters_and_logs()，则计数器已被重置。
# 如果只想运行 Mem0 部分，请取消注释下一行:
# reset_all_token_counters_and_logs()

# 清除之前的 Mem0 运行日志（如果有）（此处不重置 raw_run_log）
global mem0_run_log
mem0_run_log = []

# 设置 Mem0 运行和记忆存储操作的详细程度
VERBOSE_MEM0_RUN = False  # 设置为 True 获取详细的 Mem0 运行日志
mem0_memory_store.verbose = VERBOSE_MEM0_RUN

# 使用对话脚本运行 Mem0 驱动的对话方法
run_mem0_approach_conversation(conversation_script, mem0_memory_store)

# 收集每个子任务和整体的最终 Mem0 令牌计数
final_mem0_conv_prompt_tokens = total_prompt_tokens_mem0_conversation
final_mem0_conv_completion_tokens = total_completion_tokens_mem0_conversation
final_mem0_extr_prompt_tokens = total_prompt_tokens_mem0_extraction
final_mem0_extr_completion_tokens = total_completion_tokens_mem0_extraction
final_mem0_upd_prompt_tokens = total_prompt_tokens_mem0_update
final_mem0_upd_completion_tokens = total_completion_tokens_mem0_update

# 计算 Mem0 总体提示、补全和总令牌数
final_mem0_overall_prompt_tokens = (
    final_mem0_conv_prompt_tokens + final_mem0_extr_prompt_tokens + final_mem0_upd_prompt_tokens
)
final_mem0_overall_completion_tokens = (
    final_mem0_conv_completion_tokens + final_mem0_extr_completion_tokens + final_mem0_upd_completion_tokens
)
final_mem0_overall_total_tokens = (
    final_mem0_overall_prompt_tokens + final_mem0_overall_completion_tokens
)

print("Mem0 方法运行完成。")

In [ ]:
# 为比较分析 DataFrame 准备数据字典
# 每个键是一列: 'Metric', 'Raw Approach', 'Mem0 Approach'
data = {
    'Metric': [
        'Prompt Tokens',                  # 使用的总提示令牌（原始 vs Mem0 总体）
        'Completion Tokens',              # 使用的总补全令牌（原始 vs Mem0 总体）
        'Total Tokens',                   # 总令牌（提示 + 补全）
        '',                               # 为可读性分隔行
        'Mem0: Conversational Prompt',    # Mem0: 对话（查询）轮次的提示令牌
        'Mem0: Conversational Completion',# Mem0: 对话（查询）轮次的补全令牌
        'Mem0: Extraction Prompt',        # Mem0: 提取子任务的提示令牌
        'Mem0: Extraction Completion',    # Mem0: 提取子任务的补全令牌
        'Mem0: Update Logic Prompt',      # Mem0: 更新逻辑子任务的提示令牌
        'Mem0: Update Logic Completion'   # Mem0: 更新逻辑子任务的补全令牌
    ],
    'Raw Approach': [
        final_raw_prompt_tokens,          # 原始方法: 总提示令牌
        final_raw_completion_tokens,      # 原始方法: 总补全令牌
        final_raw_total_tokens,           # 原始方法: 总令牌
        '',                              # 分隔符（空白）
        '-',                             # 原始方法不适用
        '-',                             # 原始方法不适用
        '-',                             # 原始方法不适用
        '-',                             # 原始方法不适用
        '-',                             # 原始方法不适用
        '-'                              # 原始方法不适用
    ],
    'Mem0 Approach': [
        final_mem0_overall_prompt_tokens,     # Mem0: 总体提示令牌（所有子任务之和）
        final_mem0_overall_completion_tokens, # Mem0: 总体补全令牌（所有子任务之和）
        final_mem0_overall_total_tokens,      # Mem0: 总体总令牌
        '',                                   # 分隔符（空白）
        final_mem0_conv_prompt_tokens,        # Mem0: 对话提示令牌
        final_mem0_conv_completion_tokens,    # Mem0: 对话补全令牌
        final_mem0_extr_prompt_tokens,        # Mem0: 提取提示令牌
        final_mem0_extr_completion_tokens,    # Mem0: 提取补全令牌
        final_mem0_upd_prompt_tokens,         # Mem0: 更新逻辑提示令牌
        final_mem0_upd_completion_tokens      # Mem0: 更新逻辑补全令牌
    ]
}

In [ ]:
# 使用准备好的数据字典创建比较分析的 DataFrame
comparison_df = pd.DataFrame(data)

# 基于令牌使用情况打印分析摘要
print("\n--- 分析 --- ")

# 情况 1: 两种方法的令牌数均为零（可能是错误或不完整运行）
if final_raw_total_tokens == 0 and final_mem0_overall_total_tokens == 0:
    print("令牌计数均为零。确保运行成功完成并进行了 API 调用。")

# 情况 2: Mem0 方法使用的令牌比原始方法少（令牌节省）
elif final_mem0_overall_total_tokens < final_raw_total_tokens:
    savings = final_raw_total_tokens - final_mem0_overall_total_tokens
    percentage_savings = (savings / final_raw_total_tokens) * 100 if final_raw_total_tokens > 0 else 0
    print(f"Mem0 更令牌高效，相比原始方法节省了 {savings} 个令牌（{percentage_savings:.2f}%）。")

# 情况 3: 原始方法使用的令牌比 Mem0 少（Mem0 的令牌开销）
elif final_raw_total_tokens < final_mem0_overall_total_tokens:
    overhead = final_mem0_overall_total_tokens - final_raw_total_tokens
    percentage_overhead = (overhead / final_raw_total_tokens) * 100 if final_raw_total_tokens > 0 else float('inf')
    print(f"原始方法更令牌高效。Mem0 有 {overhead} 个令牌的开销（{percentage_overhead:.2f}%）。")
    print("（由于 Mem0 的提取/更新成本，这在短对话中是预期的。）")

# 情况 4: 两种方法使用大致相同的令牌数
else:
    print(f"两种方法使用约 {final_mem0_overall_total_tokens} 个相同令牌。")

# 关于 Mem0 在长对话中优势的一般说明
print("\nMem0 的关键优势: 由于查询提示大小稳定，在*更长*对话中的令牌效率。")

In [ ]:
# 向比较 DataFrame 添加新列 'Percentage Difference'。
# 对于每一行，如果 'Raw Approach' 和 'Mem0 Approach' 都是整数且 'Raw Approach' 不为零，
# 计算百分比差异为:
#   -((Mem0 Approach - Raw Approach) / Raw Approach) * 100
# 这显示了令牌节省的百分比（正数表示 Mem0 使用更少令牌）。
# 如果不适用，则设置为 None。
comparison_df['Percentage Difference'] = comparison_df.apply(
    lambda row: (
        -(row['Mem0 Approach'] - row['Raw Approach']) / row['Raw Approach'] * 100
        if isinstance(row['Raw Approach'], int) and isinstance(row['Mem0 Approach'], int) and row['Raw Approach'] != 0
        else None
    ),
    axis=1
)

In [ ]:
comparison_df

## 6. 预期结果讨论和进一步改进

在使用强大的 LLM 运行笔记本后：

**令牌效率**:
*   对于短对话，Mem0 的提取/更新开销可能使其总令牌数与原始方法相当或略高。然而，对于更长的对话，Mem0 应该变得显著更令牌高效，因为其查询提示大小（查询 + K 个检索的记忆）保持稳定，而原始方法的提示（完整历史）线性增长。

**响应质量和上下文处理**:
*   **原始方法**: 对于强大的 LLM 可能表现良好，但存在"中间迷失"问题、近因偏差以及在非常长的上下文中处理冲突/更新信息的困难。
*   **Mem0 方法**: 旨在使用相关检索的记忆提供集中、准确的响应。`UPDATE` 机制是处理演变信息的关键。质量取决于准确的提取和更新逻辑。

**本笔记本 Mem0 的挑战和改进**:
*   **LLM 依赖性**: Mem0 的内部操作（提取、更新决策）高度依赖于 `LLM_MODEL` 的能力。
*   **提示工程**: 提取和更新的提示至关重要，可以不断改进。
*   **错误处理**: 更健壮的 LLM JSON 输出解析和错误恢复。
*   **完整 Mem0 功能**: 实现 `DELETE`、对话摘要 `S` 和图记忆（`Mem0g`）将更接近论文。

本笔记本作为基础探索。生产就绪的系统需要更多工程，并可能需要对记忆子任务进行微调模型。